In [1]:
import os
import numpy as np
import qml
import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/hitp/')
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')
import prepare_calculations
import explore_qml_data as eqd

In [ ]:
comp = qml.Compound(xyz='/home/misa/datasets/qm9/dsgdb9nsd_000930.xyz')
charge = 0
box = 20.0
gpts = 350
els = comp.atomtypes
coords = comp.coordinates
write_path = '/home/misa/projects/Atomic-Energies/data/test.inp'
prepare_calculations.write_input(els, charge, coords, gpts, box, write_path, template_path='/home/misa/projects/Atomic-Energies/data/cpmd_params_template.inp', debug = False)

In [4]:
selected_molecules = [
 '/home/misa/datasets/qm9/dsgdb9nsd_000940.xyz',
 '/home/misa/datasets/qm9/dsgdb9nsd_000949.xyz',
 '/home/misa/datasets/qm9/dsgdb9nsd_002102.xyz',
 '/home/misa/datasets/qm9/dsgdb9nsd_002110.xyz',
 '/home/misa/datasets/qm9/dsgdb9nsd_000001.xyz',
 '/home/misa/datasets/qm9/dsgdb9nsd_000007.xyz',
 '/home/misa/datasets/qm9/dsgdb9nsd_000013.xyz',
 '/home/misa/datasets/qm9/dsgdb9nsd_000039.xyz',
 '/home/misa/datasets/qm9/dsgdb9nsd_000133.xyz',
 '/home/misa/datasets/qm9/dsgdb9nsd_000543.xyz']

In [6]:
for s in selected_molecules:
    s.split('.')[0].split('/')[-1]

dsgdb9nsd_000940
dsgdb9nsd_000949
dsgdb9nsd_002102
dsgdb9nsd_002110
dsgdb9nsd_000001
dsgdb9nsd_000007
dsgdb9nsd_000013
dsgdb9nsd_000039
dsgdb9nsd_000133
dsgdb9nsd_000543


In [7]:
for s in selected_molecules:
    path_to_xyz = s#'/home/misa/datasets/qm9/dsgdb9nsd_000930.xyz'
    compound = s.split('.')[0].split('/')[-1]
    compound_path = f'/home/misa/projects/Atomic-Energies/data/ueg_reference/SCF_calculations/{compound}/'
    lambda_values = np.array([0.2, 0.4, 0.6, 0.8, 1.0])

    comp = qml.Compound(xyz=path_to_xyz)
    atomsymbols = comp.atomtypes
    nuc_charges = comp.nuclear_charges

    # calculation parameters (independent of lambda value)
    num_ve = eqd.get_num_val_elec(nuc_charges) # get number of ve
    boxsize = prepare_calculations.get_boxsize(num_ve) # get boxsize
    num_gpts_lower, num_gpts_higher = prepare_calculations.get_gpts(num_ve) # get gridpoints
    num_gpts = num_gpts_higher

    # shift molecule to center of the box
    coords_initial = comp.coordinates
    coords = eqd.shift2center(coords_initial, np.array([boxsize, boxsize, boxsize])/2)

    # get correct lambda value
    for lam_val in lambda_values:
        new_lambda, scaled_ve = prepare_calculations.get_lambda(lam_val, num_ve)
        # scaled_ve is number of electrons added from pseudopotential file, the remaining electrons must be added in form of a negative charge
        charge = scaled_ve - num_ve # write input

        # create directory if necessary
        if scaled_ve < 10:
            scaled_ve_str = '0'+str(scaled_ve)
        else:
            scaled_ve_str = str(scaled_ve)
        work_dir = os.path.join(compound_path, f've_{scaled_ve_str}/')
        os.makedirs(work_dir, exist_ok=True)

        # generate input file
        input_path = os.path.join(work_dir, 'run.inp')
        prepare_calculations.write_input(atomsymbols, charge, coords, num_gpts, boxsize, input_path, '/home/misa/projects/Atomic-Energies/data/cpmd_params_template.inp', debug = False)

        # generate pp-files
        prepare_calculations.write_pp_files_compound(comp, new_lambda, work_dir, pp_dir='/home/misa/software/PP_LIBRARY/', pp_type='_SG_LDA')

In [ ]:
comp.natypes.keys()